In [56]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import re

In [57]:

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def remove_url(text):
    return re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)

def remove_identity(text):
    return re.sub(r'\n(Biro|Bidang|Kabinet|Divisi|Departemen|Himpunan|#|Line|Twitter|Instagram|Facebook|Youtube|Ask fm|Wa|WA|@).*', '\n\n', text)

In [58]:
separator = "<|endoftext|>"

In [59]:
list_caption = []
for file in os.listdir("Bahan Scraping"):
    with open("Bahan Scraping/"+file) as fp:
        soup = BeautifulSoup(fp, 'html.parser')
        list_caption.extend( [i.get('alt') for i in soup.find_all('img') 
                            if re.search(r'\[(.*?)\]', i.get('alt',""))]) 
list_caption = list(dict.fromkeys(list_caption))

In [60]:
df = pd.DataFrame(list_caption, columns=['raw_caption'])


#title is inside bracket
df['title'] = df['raw_caption'].str.extract(r'\[(.*?)\]', expand=False)

#Remove title from caption, remove leading newline, remove emoji
df['content'] = df['raw_caption'].apply(lambda x: deEmojify(x)).apply(lambda x: remove_url(x)).apply(lambda x: remove_identity(x)).str.split(r'\[.*?\]').apply(lambda x: x[-1]).apply(lambda x: x.lstrip())

df['processed'] = df.apply(lambda x: x['title']+" - "+x['content'], axis=1)


In [61]:
df.tail(10)['processed'].values[-10]

'SELAMAT DATANG GAMADA - Jika tidak ada yang dikorbankan, maka bukanlah suatu perjuangan.\nJika berdiri tegak tanpa ujian, maka tidaklah dinamakan ketangguhan. \nUntuk semua perjuangan, keringat, dan doa-doamu. \nSelamat datang Mahasiswa Baru Universitas Gadjah Mada 2016 yang telah diterima melalui jalur SBMPTN. Selamat bergabung dalam keluarga besar kampus kerakyatan. Selamat berkarya untuk Indonesia yang lebih baik. \nSalam Gadjah Mada Muda!\n\nInfo lebih lanjut :\n\n\n\n\n\n\n\n\n\n\n\n'

In [62]:
df['processed'].to_csv("list_caption.csv", index=False)